Notebook này sẽ triển khai Python đơn giản của mô hình ReAct cho LLMs, với mong muốn cho phép nó truy cập các tool, có thể thực hiện gọi API, chạy code, ... nói chung là phá vỡ các ràng buộc của môi trường (environment) ban đầu.

<p style="text-align: center;">
  <img src="https://react-lm.github.io/files/diagram.png" width="700" height="auto" />
</p>


Mô hình ReAct (Reason + Act) được mô tả trong [ReAct Paper](https://react-lm.github.io/). Đây là một mô hình mà triển khai các action (hành động) bổ sung mà một LLM có thể thực hiện - ví dụ như tìm kiếm trên internet (wikipedia) hoặc thực hiện tính toán - và sau đó dạy nó cách request thực hiện những action đó, rồi đưa kết quả của chúng trở lại LLM.

<p style="text-align: center;">
  <img src="https://substackcdn.com/image/fetch/w_1456,c_limit,f_auto,q_auto:good,fl_progressive:steep/https%3A%2F%2Fsubstack-post-media.s3.amazonaws.com%2Fpublic%2Fimages%2F6b27a8a7-8f67-4558-a3f4-44bf512e6c92_1766x812.gif" width="700" height="auto" />
</p>


Giờ mình sẽ thực hiện một số chức năng, action đơn giản như : 

- wikipedia: <từ khóa tìm kiếm> - tìm kiếm trên Wikipedia và trả về đoạn trích của kết quả đầu tiên
- calculate: <biểu thức> - đánh giá biểu thức bằng cách sử dụng hàm eval() của Python (lưu ý: hàm eval() sẽ nguy hiểm nếu sử dụng với chuỗi không đáng tin cậy)
- search_advanced: <từ khóa tìm kiếm> - tìm kiếm trên Google sử dụng API của Tavily

Lưu ý : Trong notebook này mình sẽ sử dụng API của Gemini chat completion vì nó đang hỗ trợ miễn phí, đã được hỗ trợ tưởng thích với thư viện OpenAI, nhưng nếu bạn muốn sử dụng mô hình của OpenAI, bạn có thể thay thế bằng API của OpenAI và bỏ qua `base_url` (mình sẽ chú thích ở dưới).

In [1]:
from openai import OpenAI
import re
import httpx
import os
from tavily import TavilyClient # tavily_search hỗ trợ tìm kiếm thông tin từ các trang web khá mạnh, https://pypi.org/project/tavily-python/

In [6]:
tavily_client = TavilyClient()
response = tavily_client.search("Leo Messi là ai?",max_results=2,include_answer=True)
print(response)

{'query': 'Leo Messi là ai?', 'follow_up_questions': None, 'answer': "Lionel Messi is a professional Argentine footballer who plays as a forward for Major League Soccer club Inter Miami CF and the Argentina national team. He has won the Ballon d'Or a record eight times and was named FIFA's Best Player of the Year in 2023. In 2023, he also won the World Cup with Argentina and the Ligue 1 title with Paris Saint-Germain.", 'images': [], 'results': [{'title': 'Lionel Messi - Wikipedia', 'url': 'https://en.wikipedia.org/wiki/Lionel_Messi', 'content': "Messi began the 2011–12 season winning both the Spanish and European Super Cups trophies.[91][92] At the close of the year, he won the FIFA Club World Cup and earned the Golden Ball for a second time.[93] For his efforts in 2011, he received the FIFA Ballon d'Or, becoming only the fourth player in history to win the Ballon d'Or three times,[94] and the inaugural UEFA Best Player in Europe Award.[95] During the year 2012, Messi became the secon

In [15]:
import json
from pygments import highlight, lexers, formatters


# pretty print JSON with syntax highlighting
formatted_json = json.dumps(response, indent=4)
colorful_json = highlight(formatted_json,
                          lexers.JsonLexer(),
                          formatters.TerminalFormatter())

print(colorful_json)


{
    "query": "Leo Messi l\u00e0 ai?",
    "follow_up_questions": null,
    "answer": "Lionel Messi is a professional Argentine footballer who plays as a forward for Major League Soccer club Inter Miami CF and the Argentina national team. He has won the Ballon d'Or a record eight times and was named FIFA's Best Player of the Year in 2023. In 2023, he also won the World Cup with Argentina and the Ligue 1 title with Paris Saint-Germain.",
    "images": [],
    "results": [
        {
            "title": "Lionel Messi - Wikipedia",
            "url": "https://en.wikipedia.org/wiki/Lionel_Messi",
            "content": "Messi began the 2011\u201312 season winning both the Spanish and European Super Cups trophies.[91][92] At the close of the year, he won the FIFA Club World Cup and earned the Golden Ball for a second time.[93] For his efforts in 2011, he received the FIFA Ballon d'Or, becoming only the fourth player in history to win the Ballon d'Or three times,[94] and the inaugural UEFA 

In [7]:
response['answer']

"Lionel Messi is a professional Argentine footballer who plays as a forward for Major League Soccer club Inter Miami CF and the Argentina national team. He has won the Ballon d'Or a record eight times and was named FIFA's Best Player of the Year in 2023. In 2023, he also won the World Cup with Argentina and the Ligue 1 title with Paris Saint-Germain."

In [21]:
API_KEY = os.getenv("GOOGLE_API_KEY") # nếu bạn sử dụng OPENAI API thì thay bằng OPENAI_API_KEY
BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/" # Nếu bạn sử dụng OPENAI API thì có thể để None
MODEL = "gemini-2.0-flash" # Nếu bạn sử dụng OPENAI API thì thay bằng tên model của bạn

In [22]:
client = OpenAI(
  api_key=API_KEY,
  base_url=BASE_URL,
)

In [33]:
class ChatBot_ReAct:
    def __init__(self, system: str = "", messages: list = []):
        self.system = system
        self.messages = messages
        if self.system:
            self.messages.append({"role": "system", "content": system})
    
    def __call__(self, message: str) -> str:
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result, self.messages
    
    def execute(self) -> str:
        completion = client.chat.completions.create(model=MODEL, messages=self.messages, temperature = 0.0)
        return completion.choices[0].message.content

In [34]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

You will final answer in the language of the user, for example the user asks Vietnamese, you must answer in Vietnamese

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

wikipedia:
e.g. wikipedia: Django
Returns a summary from searching Wikipedia

search_advanced:
e.g. search_advanced: New news update about Messi
Returns a summary from searching Internet, using Tavily, a powerful search engine that can search information from many websites

Always look for everything on Wikipedia if you have the opportunity to do so. If difficult, use search_advanced

Example session:

Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Action: wikipedia: France
PAUSE

You will be called again with this:

Observation: France is a country. The capital is Paris.

You then output:

Answer: The capital of France is Paris.
""".strip()

In [35]:
def wikipedia(q):
    content_en = httpx.get("https://en.wikipedia.org/w/api.php", params={
        "action": "query",
        "list": "search",
        "srsearch": q,
        "format": "json"
    }).json()["query"]["search"][0]["snippet"]

    content_vi = httpx.get("https://vi.wikipedia.org/w/api.php", params={
        "action": "query",
        "list": "search",
        "srsearch": q,
        "format": "json"
    }).json()["query"]["search"][0]["snippet"]
    
    return content_en+"\n"+content_vi

def calculate(what):
    return eval(what)

def search_advanced(query):
    response = tavily_client.search(query,max_results=1, exclude_domains=["wikipedia.org"], include_answer = True) # exclude_domains để loại bỏ các trang web không cần thiết
    # return str({key: value for key, value in response['results'][0].items() if key not in ['score', 'raw_content']})
    return response['answer']

known_actions = {
    "wikipedia": wikipedia,
    "calculate": calculate,
    "search_advanced": search_advanced,
}

In [36]:
action_re = re.compile('^Action: (\w+): (.*)$')

def query(question: str,history: list = None)-> list:
    max_turns = 5
    if history is None:
        history = []

    i = 0
    bot = ChatBot_ReAct(prompt,history)
    next_prompt = question
    while i < max_turns:
        i += 1
        result, messages = bot(next_prompt)
        print(result)
        actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]
        if actions:
            # There is an action to run
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception("Unknown action: {}: {}".format(action, action_input))
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = "Observation: {}".format(observation)
        else:
            print("Answer:", result)
            try :
                return result.split("Answer: ")[1].strip()
            except:
                return result

In [37]:
query("Hello")

Thought: The user greeted me. I should respond in a friendly manner.
Action: None
PAUSE
Observation: None
Answer: Hello! How can I help you today?

Answer: Thought: The user greeted me. I should respond in a friendly manner.
Action: None
PAUSE
Observation: None
Answer: Hello! How can I help you today?



'Hello! How can I help you today?'

In [47]:
q1 = query("Thủ đô của nước Việt Nam là gì")

Thought: Tôi nên tìm kiếm thông tin về Việt Nam trên Wikipedia để tìm thủ đô.
Action: wikipedia: Việt Nam
PAUSE

 -- running wikipedia Việt Nam
Observation: 699 square kilometres. <span class="searchmatch">Vietnamese</span>: <span class="searchmatch">Việt</span> <span class="searchmatch">Nam</span> [vîət <span class="searchmatch">nāːm</span>] The spelling &quot;<span class="searchmatch">Viet</span> <span class="searchmatch">Nam</span>&quot; or the full marked-<span class="searchmatch">Vietnamese</span> form &quot;<span class="searchmatch">Việt</span> <span class="searchmatch">Nam</span>&quot; is sometimes used in English
<span class="searchmatch">Việt</span> <span class="searchmatch">Nam</span>, quốc hiệu là Cộng hòa xã hội chủ nghĩa <span class="searchmatch">Việt</span> <span class="searchmatch">Nam</span>, là một quốc gia nằm ở cực Đông của bán đảo Đông Dương thuộc khu vực Đông <span class="searchmatch">Nam</span> Á, giáp với Lào
Thought: Kết quả tìm kiếm trên Wikipedia không trực ti

In [49]:
# q2 = query("chi tiết thủ đô đó", history=q1)

In [12]:
# query("tôi có 10 quả táo, tôi cho bạn 3 quả, tôi còn bao nhiêu quả táo?")

In [19]:
import gradio as gr

In [45]:
def chat_interface_handler(message, history):
    """
    Hàm xử lý tin nhắn cho ChatInterface
    
    Args:
        message (str): Tin nhắn từ người dùng
        history (list): Lịch sử chat [(user_message, bot_message), ...]
    
    Returns:
        str: Phản hồi của bot
    """
    # Chuyển đổi định dạng lịch sử chat từ Gradio sang định dạng của ChatBot_ReAct
    bot_messages = []
    for user_msg, bot_msg in history:
        bot_messages.append({"role": "user", "content": user_msg})
        bot_messages.append({"role": "assistant", "content": bot_msg})
    
    try:
        # Gọi hàm query với tin nhắn hiện tại và lịch sử
        answer = query(message, bot_messages)
        return answer
    except Exception as e:
        # Xử lý lỗi
        return f"Error: {str(e)}"

# Tạo giao diện ChatInterface
demo = gr.ChatInterface(
    fn=chat_interface_handler,
    title="ReAct Agent",
    description="Đặt câu hỏi và Chatbot sẽ sử dụng các công cụ để tìm câu trả lời.",
    examples=["Lionel Messi là ai?", "Tìm hiểu về AI", "50 USD bằng bao nhiêu VND?"],
    theme="ocean",
)

# Khởi chạy ứng dụng
if __name__ == "__main__":
    demo.launch(share=False)
    

c:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\gradio\chat_interface.py:334: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7870

To create a public link, set `share=True` in `launch()`.


Thought: Tôi cần tìm kiếm mô hình mới nhất của OpenAI. Tôi sẽ sử dụng công cụ tìm kiếm để tìm thông tin này.
Action: search_advanced: mô hình mới nhất của OpenAI
PAUSE

 -- running search_advanced mô hình mới nhất của OpenAI
Observation: {'title': 'OpenAI ra mắt GPT-4.5: Mô hình lớn nhất và am hiểu nhất từ trước đến ...', 'url': 'https://vatvostudio.vn/openai-ra-mat-gpt-4-5/', 'content': 'OpenAI vừa chính thức ra mắt GPT-4.5, mô hình ngôn ngữ AI mới nhất và lớn nhất của mình. Tuy được quảng bá là "mô hình am hiểu nhất từ trước đến nay", OpenAI cũng đồng thời cảnh báo GPT-4.5 không phải là một mô hình AI đột phá và hiệu năng có thể chưa sánh bằng các mô hình nghiên cứu nội bộ như o1 hay'}
Answer: Theo kết quả tìm kiếm, OpenAI vừa ra mắt GPT-4.5, mô hình ngôn ngữ AI mới nhất và lớn nhất của họ.

Answer: Answer: Theo kết quả tìm kiếm, OpenAI vừa ra mắt GPT-4.5, mô hình ngôn ngữ AI mới nhất và lớn nhất của họ.

Thought: I should use search_advanced to find more details about GPT-4.5.
Actio

In [50]:
demo.close() # Đóng ứng dụng

Closing server running on port: 7870
